# Important Trace With Nsight Systems
---

This notebook exposes you to important insights with multi-GPU profiling using four GPUs. It includes trace on NCCL AllReduce, NCCL Broadcast, NVLink, CUDA API, CUDA GPU Kernel, Mixed precision, GPU Gap, and CUDA Async With Memcpy Pageable Memory. 

### NCCL AllReduce Trace

Before we start the trace, let's refresh our knowledge of the `Allreduce` operation. AllReduce allows all the nodes/GPUs to get the same aggregated value. The All-Reduce function allows all the workers to get the aggregated gradients from all the worker nodes. This gradient aggregation is the model synchronization procedure in the All-Reduce architecture. It guarantees that all the workers use the same gradient to update the model in the current training iteration. All-Reduce uses all-to-all communication. Every worker needs to send their value to all the other workers. The All-reduce function is implemented as `Ring All-Reduce` using NVIDIA NCCL. Since our DDP program was profiled on a single node with 4 GPUs, the report screenshot below shows NCCL kernel launch for Ring AllReduce Sum operation using `FP32`. The tooltip on the kernel for `GPU0` detailed the size of shared memory, threads, grid, and blocks and also included latency and kernel total execution time. It is important to note that AllReduce Kernels were launched by all the GPUs on the current timeline `(0.45s ~ 0.49s)` except for `GPU3`, which differs slightly. This occurrence may likely impact the `workers` synchronization process as there will be a wait for GPU3 in `Ring AllReduce Sum operation` sync for all GPUs.  


<center><img src="images/GPU0-ncclallreduce.png" width="850px" height="850px" alt-text="workflow"/></center>

<center><img src="images/GPU1-ncclallreduce.png" width="850px" height="850px" alt-text="workflow"/></center>

<center><img src="images/GPU2-3-ncclallreduce.png" width="850px" height="850px" alt-text="workflow"/></center>
<br/>

One way to trace the cause of the delay in GPU3 is to check if there was a delay in the data loader/data copy to the device (HtoD). Also, to look further into processes running on the CPU on the timline.   

### NCCL Broadcast Trace

After an AllReduce operation has been completed, the master worker has to broadcast the aggregated value to all other workers or devices for an update. The report screenshot below shows `GPU0` as the master worker performing `ncclBroadcast` task through the `ncclDveKernel_Broadcast_RING_LL` kernel. Meanwhile GPU1 and GPU2 are busy on `memcpy HtoD` (copy from Host to device) while GPU3 performs a backward pass.

<center><img src="images/ncclBroadcast012.png" width="900px" height="900px" alt-text="workflow"/></center>

<center><img src="images/ncclBroadcast3.png" width="850px" height="850px" alt-text="workflow"/></center>


### NVLink Trace

Another important trace in a multi-GPU environment is the data transfer speed between devices. This process heavily relies on the NVLink Bandwidth for both transmission (Tx) and receiving (Rx) measured in throughput percentage. To check for NVLink Tx and Rx on the report, click on any GPUs. You will see the `GPU Metrics (10kHz)` click on it to drop-down, and there you will see the `NVLink Rx Bandwidth` and `NVLink Tx Bandwidth.` Hover your mouse on it to see the popup box that displays the `throughput %` as shown in the screenshot of the report for the NVLink transmission below.

<center><img src="images/nvlink-Tx.png" width="850px" height="850px" alt-text="workflow"/></center>

### Stats System View

The Stats Systems View is a feature of Nsight Systems that provides summaries on time spent by CUDA Kernels, Memory Operations, GPU trace, etc. You can access this feature by switching from the default `Event View` to select `Stats Systems View` in the dropdown. From the top box in the report screenshot below, we deduce that `cudaStreamSynchronize` took 62% of the time of all the CUDA API calls. This implies that more operations that require synchronization frequently occur. Such a process could be allreduce-sum and broadcast Kernel calls. You may have to investigate further by checking the CUDA kernel summary to ascertain the assumption. In the bottom green box of the screenshot, We see that `ncclDevKernel_AllReduce_Sum_f32_RING_LL` executes frequently (73%), and this supports our assumption of frequent `cudaStreamSynchronize` call. More importantly, you can also verify the use of mixed-precision (FP16 & FP32) operations that were enabled in the previous notebook, as indicated by red arrows. You can explore more based on the insight you want to draw or see.


<center><img src="images/SSV.png" width="850px" height="850px" alt-text="workflow"/></center>

### Expert System View

Just like the way you switch from `Event View` to `Stats Systems View,` you can also change the view to `Expert System View.` The Expert System View feature provides an easy-to-understand message or summarized details on `CUDA Async Memcpy with Pageable Memory,` `CUDA Synchronous Memcpy,` `CUDA Synchronous Memset,` `CUDA Synchronization APIs,` `GPU Gaps,` `GPU Utilization,` and others. In the report screenshot of our DDP program, we examine the state of Pageable memory, which indicates no issues, and also GPU Gaps, which reported no issues with GPU utilization and not more the 500ms GPU idle time occurred. Please feel free to explore more based on your application needs.


<center><img src="images/ESV.png" width="850px" height="850px" alt-text="workflow"/></center>

We have come to the end of the lab, and we hope you learned a lot to develop yourself and apply the knowledge in your project.  

## Links and Resources


[NVIDIA Nsight Systems](https://developer.nvidia.com/nsight-systems)


**NOTE**: To be able to see the profiler output, please download the latest version of NVIDIA Nsight Systems from [here](https://developer.nvidia.com/nsight-systems/get-started).


You can also get resources from [Open Hackathons technical resource page](https://www.openhackathons.org/s/technical-resources)


--- 

## Licensing 

Copyright © 2025 OpenACC-Standard.org. This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). These materials may include references to hardware and software developed by other entities; all applicable licensing and copyrights apply.